In [1]:
#Je achete le boeuf!
#plotting imports
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# ignore warnings
import warnings
warnings.filterwarnings("ignore")


#modeling imports
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz

#pipeline modules
import acquire
import explore
import prepare

In [2]:
df = prepare.prep_titanic()

In [3]:
df.dropna(inplace=True)
df.isnull().sum()

passenger_id    0
survived        0
pclass          0
sex             0
age             0
sibsp           0
parch           0
fare            0
embarked        0
class           0
embark_town     0
alone           0
dtype: int64

In [4]:
#DATA IS ACQUIRED AND PREPARED.
#LETS DO THE 4 WAY SPLIT OF X-y, and each of those are split into train and test.
X=df[['pclass','age','fare','sibsp','parch']]
y=df[['survived']]
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.30,random_state=123)

# THIS IS THE LOGREG PART

In [ ]:
logit = LogisticRegression(C=1,class_weight={1:2},random_state=123,solver='saga')

In [ ]:
logit.fit(X_train, y_train)

In [ ]:
x = logit.coef_

In [ ]:
print(f"The coefficients for the predictors are {x}.")
print(f"The y-intercept for the model is {logit.intercept_}.")

In [ ]:
#An array of predicted labels of the target variable generated by fitting the Xtrain and ytrain to LogReg model
#Some are right, some are wrong.
y_pred = logit.predict(X_train)

In [ ]:
y_pred_proba = logit.predict_proba(X_train)

In [ ]:
blob = confusion_matrix(y_train, y_pred)

In [ ]:
#That little insignificant 2x2 matrix is the confusion matrix. 
print(blob)

### Looks like terrible precision but okay recall.

In [ ]:
cr=(classification_report(y_train,y_pred))

In [ ]:
score = logit.score(X_test,y_test)

### Why do all that work when sklearn.metrics.classification_report will do it for us

In [ ]:
print(cr)

In [ ]:
true_negative = blob[0][0]
true_positive = blob[1][1]
false_negative = blob[1][0]
false_positive = blob[0][1]

In [ ]:
accuracy=((true_negative+true_positive)/len(y_train))
#all the correctly predicted positives divided by that AND the ones that were predicted negative but shouldn't have
recall=true_positive/(true_positive+false_negative)
#All the corretly predicted positives, divied by that and the ones that the model was too eager to predict as positive
precision=true_positive/(true_positive+false_positive)
#Works horizontally along the actual positive. FP/FP+TN
false_positive_rate=false_positive/(false_positive+true_negative)

In [ ]:
print(f"The accuracy is {accuracy}")
print(f"The recall is {recall}")
print(f"The precision is {precision}")

In [ ]:
def do_the_logRegression(X_train, y_train, my_solver):
    logit = LogisticRegression(C=1,class_weight={1:2},random_state=123,solver=my_solver)
    logit.fit(X_train, y_train)
    y_pred = logit.predict(X_train)
    y_pred_proba=logit.predict_proba(X_train)
    score = logit.score(X_train,y_train)
    return score

In [ ]:
do_the_logRegression(X_train, y_train, 'lbfgs')

In [ ]:
solvers = ["lbfgs", "liblinear", "sag", "saga", "newton-cg"]

In [ ]:
for i in solvers:
    print(f"The accuracy for {i} as the solver is {do_the_logRegression(X_train, y_train, i)}")

## DECISION TREES

In [5]:
# Fit the decision tree classifier to your training sample and transform (i.e. make predictions on the training sample)
# The Ol' 1-2-3
# 1
clf = DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=123)

In [6]:
# 2
clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=3,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=123, splitter='best')

In [14]:
# 3
y_pred = clf.predict(X_train)
y_pred_proba = clf.predict_proba(X_train)

In [13]:
# type(y_pred)
# y_pred[:6]
# y_train[:6]

In [15]:
conf_matx = confusion_matrix(y_train, y_pred)

In [16]:
print(conf_matx)

[[274  19]
 [123  83]]


In [18]:
type(conf_matx)

# conf_matx.transpose()

array([[274, 123],
       [ 19,  83]])